In [1]:
import os
import functions as fn
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from traffic.core import Traffic

### Fetching data in daily packages

In [2]:
# Fetching data in daily chunks
dates = pd.date_range('2022-01-01', '2022-12-01', freq='D', tz="UTC")
area = fn.query_bound((47.48891, 8.85743), 20, 20)
path_raw = 'data/rectangle_1/2022/raw'
path_combined = 'data/rectangle_1/2022/combined'

fn.download_adsb_para(dates, path_raw, fn.query_bound((47.48891, 8.85743), 20, 20))

# Combining all the data into one file
check_file = Path(f"{path_combined}/combined.parquet",
                      use_deprecated_int96_timestamps=True)
if check_file.is_file() == False:

    files = os.listdir(path_raw)

    for i in tqdm(range(len(files))):
        if i == 0:
            alldata = Traffic.from_file(f'{path_raw}/{files[i]}').data
        else:
            alldata = pd.concat([alldata, Traffic.from_file(f'{path_raw}/{files[i]}').data])

    alldata = Traffic(alldata)

    if os.path.isdir(path_combined) == False:
            os.mkdir(path_combined)

    alldata.to_parquet(f'{path_combined}/combined.parquet', use_deprecated_int96_timestamps=True)